# REDES NEURAIS ARTIFICIAIS

[Aula 3 de Deep Learning](http://bit.ly/dn-unb03) da Engenharia de Software da UnB

# Parte 1 - Pré-processamento dos Dados

### Importar as libs

In [4]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [5]:
tf.__version__

'2.3.1'

### Importar o dataset

In [6]:
dataset = pd.read_csv('Churn_Modelling.csv')
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [7]:
X = dataset.iloc[:, 3:13].values
y = dataset.iloc[:, 13].values
print("X >>\n",X)
print("y >>\n",y)

X >>
 [[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]
y >>
 [1 0 1 ... 1 1 0]


### Transformando os dados categóricos


In [8]:
# Label Encoding the "Gender" column
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])
print(X)
# One Hot Encoding the "Geography" column
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))
print(X)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]
[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


# Dividindo o dataset em conjunto de treinamento e conjunto de teste

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

# Feature Scaling

In [10]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [11]:
np.shape(X_train)

(8000, 12)

---
# Parte 2 -Vamos construir uma ANN!


### Initializing the ANN

In [16]:
ann = tf.keras.models.Sequential()

### Adding the input layer and the first hidden layer

In [17]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

### Adding the second hidden layer

In [18]:
ann.add(tf.keras.layers.Dense(units=6, activation='sigmoid'))

### Adding the output layer

In [19]:
ann.add(tf.keras.layers.Dense(units=1, activation='tanh'))

## Part 3 - Training the ANN

### Compiling the ANN

In [20]:
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Training the ANN on the Training set

In [21]:
ann.fit(X_train, y_train, batch_size = 32, epochs = 100)

Epoch 1/100
250/250 [==============================] - 1s 4ms/step - loss: 1.1293 - accuracy: 0.7517
Epoch 2/100
250/250 [==============================] - 1s 3ms/step - loss: 0.5915 - accuracy: 0.7692
Epoch 3/100
250/250 [==============================] - 1s 3ms/step - loss: 0.5280 - accuracy: 0.7945
Epoch 4/100
250/250 [==============================] - 1s 3ms/step - loss: 0.5009 - accuracy: 0.7959
Epoch 5/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4869 - accuracy: 0.7961
Epoch 6/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4760 - accuracy: 0.7961
Epoch 7/100
250/250 [==============================] - 1s 4ms/step - loss: 0.4675 - accuracy: 0.7961
Epoch 8/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4650 - accuracy: 0.7961
Epoch 9/100
250/250 [==============================] - 1s 4ms/step - loss: 0.4605 - accuracy: 0.7962
Epoch 10/100
250/250 [==============================] - 1s 4ms/step - loss: 0.4595 - accura

250/250 [==============================] - 1s 2ms/step - loss: 0.3558 - accuracy: 0.8485
Epoch 81/100
250/250 [==============================] - 1s 2ms/step - loss: 0.3556 - accuracy: 0.8493
Epoch 82/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3562 - accuracy: 0.8493
Epoch 83/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3558 - accuracy: 0.8497
Epoch 84/100
250/250 [==============================] - 1s 4ms/step - loss: 0.3543 - accuracy: 0.8496: 0s -
Epoch 85/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3544 - accuracy: 0.8491
Epoch 86/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3536 - accuracy: 0.8500
Epoch 87/100
250/250 [==============================] - 1s 2ms/step - loss: 0.3539 - accuracy: 0.8499
Epoch 88/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3532 - accuracy: 0.8505
Epoch 89/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3525 - accu

# Parte 3 - Fazendo predições e avaliando o modelo

## Prevendo os resultados com o conjunto de testes

In [22]:
y_pred = ann.predict(X_test)
y_pred

array([[0.1759644 ],
       [0.39693552],
       [0.15200692],
       ...,
       [0.22459114],
       [0.2243645 ],
       [0.2312714 ]], dtype=float32)

In [23]:
y_pred = (y_pred > 0.5)
y_pred

array([[False],
       [False],
       [False],
       ...,
       [False],
       [False],
       [False]])

# Criando uma Confusion Matrix

In [24]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[1521   74]
 [ 208  197]]


# FIM

> Professor Diego Dorgam  
> [@diegodorgam](https://twitter.com/diegodorgam)